<a href="https://colab.research.google.com/github/thesteuber/assign03-50-steuber/blob/1-create-new-notebook-name-churn-prediction/churn_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Machine Learning - Assignment 3 "Employee Churn" 


##Purpose
Gain experience applying scikit learn to machine learning problems.

##Tasks

###1. Create a new notebook 


---


Create a new notebook named "churn-prediction", and properly document the notebook with descriptions of the tasks.

Task completed as seen in this notebook.

###2. Read in HR Data


---


Using the scikit-demo-solutions notebook as a guide, read in the HR data.

###3. Split the Data


---


Split the data for train and test.

###4. Build Pipeline


---


Build an appropriate training pipeline.

###5. Cross Validation


---


Carry out cross-validation